In [13]:
#default_exp blocs

In [32]:
import local.cg as cg


In [3]:
#export
class Operator():
    def __init__(self,input_var_names=[],namePrefix:str = None, observes:bool = False):
        self.__name__ = f"{namePrefix}_{'_'.join(input_var_names)}"
        if type(input_var_names) == str : input_var_names = [input_var_names]
        self._args = [*input_var_names] 
        if len(input_var_names) == 1: self._args = input_var_names[0].split()
        self._observes = observes
    def __call__(self,*args):
        raise NotImplemented("Operator is abstract, you must subclass it")


In [15]:
#export 
class Avg(Operator):
    """
    Compute the running average of a variable of name passed as
    `input_var_name` and output a var named "Avg_"+`input_var_name`
    
    Args:
    ========================
        * `input_var_name`: str or list<str>
        if str, content is splitted by spaces
    
    """
    def __init__(self,input_var_name):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Avg", observes = True)
        self._running_avg:float = 0
        self._running_count:int = 0
    def __call__(self,varin):
        self._running_count += 1
        self._running_avg = (varin/self._running_count) + (self._running_avg*((self._running_count-1)/self._running_count) )
        #print(f"run A {self._running_avg}")
        return self._running_avg

class Exp_avg(Operator):
    """
    Compute the running Exponential average of a variable of name passed as
    `input_var_name` and output a var named "Exp_avg_"+`input_var_name`
    
    Args:
    ========================
        * `input_var_name`: str or list<str>
        if str, content is splitted by spaces
    
    """
    def __init__(self,input_var_name, alpha = 0.9):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Exp_avg", observes = True)
        self._running_avg:float = 0
        assert(0<alpha<1)
        self._alpha : float = alpha
        self._1_minus_alpha : float = (1. - alpha)
    def __call__(self,varin):
        self._running_avg = (self._alpha * self._running_avg) + (self._1_minus_alpha * varin)
        #print(f"run EA {self._running_avg}")
        return self._running_avg    


class Acc(Operator):
    """
    Sum each value took by `input_var_name` and output a var named  "Acc_"+`input_var_name`
    
    Args:
    ========================
        * `input_var_name`: str or list<str>
        if str, content is splitted by spaces
    
    """
    def __init__(self,input_var_name):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Acc",  observes = True)
        self._running_sum:float = 0
    def __call__(self,varin):
        self._running_sum += varin
        #print(f"run Acc {self._running_sum}")
        return self._running_sum

In [5]:
#test
AvgA = Avg("A")
AccA = Acc("A")
Exp_avgA = Exp_avg("A",alpha = 0.0000001)
a,b,c = 0,0,0
for i in range(51): 
    a,b,c = AvgA(i),AccA(i),Exp_avgA(i)

print(a,b,c)

print(f"b = {50*51/2}")


25.0 1275 49.9999999
b = 1275.0


In [21]:
#export
class Linear(Operator):
    """performs gain*x + bias"""
    def __init__(self,input_var_name, gain = 1, bias =0):
        super().__init__(input_var_names = [input_var_name], namePrefix="Linear")
        self.gain, self.bias = gain, bias
    def __call__(self,x): return self.gain * x + self.bias 

class Add(Operator):
    """Adds every vars passed as imputs"""
    def __init__(self,*input_var_names) : super().__init__(*input_var_names, namePrefix="Add")
    def __call__(self,*inputs) : return sum(inputs)
    
class Mult(Operator):
    """Multiply every vars passed as imputs"""
    def __init__(self,*input_var_names) : super().__init__(*input_var_names, namePrefix="Mult")
    def __call__(self,*inputs) : 
        ret = 1.
        for inp in inputs : ret *= inp
        return ret
    
class Const(Operator):
    """always returns the same value passed as val"""
    def __init__(self, val): 
        super().__init__([], namePrefix="Const")
        self._val = val
    def __call__(self): return self._val

In [30]:
#export
class Diff(Operator):
    """
    returns the discreate differential of the input variable ponetialy over 
    `delay_in_step` timesteps
    """
    def __init__(self,input_var_name, init_val, delay_in_step:int = 1):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Diff",  observes = True)
        assert(type(delay_in_step) == int)
        from collections import deque
        self._delay_in_step =delay_in_step
        self._past_vals : deque = deque([init_val for i in range(3*delay_in_step//2)])
    def __call__(self,varin):
        self._past_vals.popleft()
        self._past_vals.append(varin)
        ret = sum(self._past_vals[:self._delay_in_step])/self._delay_in_step
        self._past_vals.popleft()
        return (varin-ret)/self._delay_in_step


class Delay(Operator):
    """
    returns the value of  the input variable delayed of
    `delay_in_step` timesteps
    """
    def __init__(self,input_var_name, init_val, delay_in_step:int = 1):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Delayed",  observes = True)
        assert(type(delay_in_step) == int)
        from collections import deque
        self._past_vals : deque = deque([init_val for i in range(delay_in_step)])
    def __call__(self,varin):
        self._past_vals.append(varin)
        return self._past_vals.popleft()

class FullReturnDelay(Operator):
    def __init__(self,input_var_name, init_val, delay_in_step:int = 1):
        super().__init__(input_var_names = [input_var_name],namePrefix = "FullReturnDelayed",  observes = True)
        assert(type(delay) == int)
        from collections import deque
        self._step = 0
        self._past_vals : deque = deque([init_val for i in range(delay_in_step)])
        self._delay_in_step  = delay_in_step
    def __call__(self,varin):
        if self._step % self._delay_in_step == 0 : 
            self._return = list(self._past_vals)
        self._step += 1
        self._past_vals.append(varin)
        self._past_vals.popleft()
        return self._return

In [ ]:
#export
class Saturation(Operator):
    """return at most Max and at least Min but varin if inbetween the two"""
    def __init__(self,input_var_name, Min, Max):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Saturation")
        self._min, self._max = Min, Max
    def __call__(self,varin): return max(self._min, min(self._max, varin))

In [28]:
#export
class Sink(Operator):
    """
    returns the value of  the input variable delayed of
    `delay_in_step` timesteps
    """
    def __init__(self, input_var_name):
        super().__init__(input_var_names = [input_var_name],namePrefix = "Sink",  observes = True)
        self._vals = []
    def __call__(self,varin): self._vals.append(varin)

In [37]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted GC.ipynb.
Converted blocs.ipynb.


In [36]:
import numpy as np
min((np.arange(3) + 1), sum([np.arange(3), np.arange(3), np.arange(3)]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:

if len(input_var_names) == 1 : input_var_names[0].split()